In [7]:
import numpy as np
import pandas as pd

df = pd.read_csv('/content/gurgaon_properties_post_feature_selection.csv')

In [8]:
df.head()

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category,price
0,1.0,7.0,3,3,0.0,0.0,1000.0,0,0,0,1.0,1.0,0.74
1,0.0,80.0,6,7,4.0,3.0,5333.0,1,0,0,2.0,1.0,5.00
2,0.0,112.0,3,2,2.0,4.0,1014.0,0,0,0,2.0,1.0,0.90
3,1.0,53.0,3,9,4.0,3.0,1935.0,0,1,1,2.0,2.0,5.15
4,0.0,39.0,3,3,3.0,4.0,1619.0,0,0,0,1.0,0.0,1.70


# firsting fitting linear reg (hence ONE)

### one hot encode -> sector, balcony, agePossession, furnishing type, luxury category, floor category


In [9]:
X = df.drop(columns=['price'])
y = df['price']

In [10]:

from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR

In [11]:
columns_to_encode = ['sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']


In [12]:
# Applying the log1p transformation to the target variable ( for giving it a stable range )
y_transformed = np.log1p(y)

In [18]:
# creating a column transformer for preprocessing

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['property_type', 'bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OneHotEncoder(drop='first',handle_unknown='ignore'), columns_to_encode)
    ],
    remainder='passthrough'
)



In [19]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', SVR(kernel='rbf'))
])

In [20]:
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['property_type', 'bedRoom',
                                                   'bathroom', 'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['sector', 'balcony',
                                                   'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category'])])),
                ('regressor', SVR())])

In [21]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [22]:
scores.mean()

np.float64(0.8853360256198124)

In [23]:

scores.std()

np.float64(0.011813970046129437)

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)


In [30]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['property_type', 'bedRoom',
                                                   'bathroom', 'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['sector', 'balcony',
                                                   'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category'])])),
                ('regressor', SVR())])

In [31]:
y_pred = pipeline.predict(X_test)

In [32]:

y_pred = np.expm1(y_pred)

In [33]:

from sklearn.metrics import mean_absolute_error
mean_absolute_error(np.expm1(y_test),y_pred)

0.5749272004440625

# according to our base model
  * it is error in carore, means while prediction ...
  * model +-57 lakh ki galti kar skta hai.
  * task is to reduce it like +-15 ( best predictor model )